# Fair Forge Generators - Groq Example

This notebook demonstrates how to use the Fair Forge generators module with **Groq Cloud** for ultra-fast synthetic test dataset generation.

## Overview

The `GroqGenerator` uses LangChain to interact with Groq's inference API, which provides extremely fast inference for open-source LLMs.

### Why Groq?
- **Speed**: Up to 10x faster than traditional cloud providers
- **Cost**: Competitive pricing for high-volume usage
- **Models**: Access to popular open-source models (Llama 3, OSS GPT)

## Setup

1. Get your free API key from [Groq Console](https://console.groq.com/)

2. Set your Groq API key as an environment variable:

```bash
export GROQ_API_KEY="your-api-key"
```

Or create a `.env` file:
```.env
GROQ_API_KEY=your-api-key
```

3. Install required dependencies:
```bash
uv venv
source .venv/bin/activate
uv pip install ".[generators-groq]" python-dotenv
uv run jupyter lab
```

**Note:** Use `.[generators-groq]` to get the correct LangChain dependencies. The base `.[generators]` group does not include `langchain-groq`.

If you're already in Jupyter and install packages, **restart the kernel** for changes to take effect.

## Imports

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

from fair_forge.generators import (
    create_groq_generator,
    create_markdown_loader,
    GroqGenerator,
    # Strategies for chunk selection
    SequentialStrategy,
    RandomSamplingStrategy,
)
from fair_forge.schemas import Dataset, Batch

# Load environment variables
load_dotenv()

print("Imports loaded successfully")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Imports loaded successfully


## Create Sample Content

Let's create a sample markdown document for testing:

In [2]:
sample_content = """# Machine Learning Fundamentals

This guide covers the basics of machine learning for beginners.

## Types of Machine Learning

Machine learning can be categorized into three main types:

### Supervised Learning
- Uses labeled training data
- Predicts outcomes based on input features
- Examples: Classification, Regression

### Unsupervised Learning
- Works with unlabeled data
- Discovers hidden patterns and structures
- Examples: Clustering, Dimensionality Reduction

### Reinforcement Learning
- Agent learns through interaction with environment
- Maximizes cumulative reward
- Examples: Game playing, Robotics

## Model Evaluation

Key metrics for evaluating ML models:

- **Accuracy**: Proportion of correct predictions
- **Precision**: True positives among predicted positives
- **Recall**: True positives among actual positives
- **F1 Score**: Harmonic mean of precision and recall

## Best Practices

1. Split data into train/validation/test sets
2. Use cross-validation for robust evaluation
3. Monitor for overfitting
4. Document your experiments
"""

# Save to file
sample_file = Path("./ml_fundamentals.md")
sample_file.write_text(sample_content)
print(f"Sample content saved to: {sample_file}")

Sample content saved to: ml_fundamentals.md


## Create Context Loader

In [3]:
# Create markdown loader
loader = create_markdown_loader(
    max_chunk_size=2000,
    header_levels=[1, 2, 3],
)

# Preview chunks
chunks = loader.load(str(sample_file))
print(f"Created {len(chunks)} chunks:\n")
for chunk in chunks:
    print(f"- {chunk.chunk_id}: {len(chunk.content)} chars")

2026-01-14 13:16:35.013 | INFO     | fair_forge.generators:create_markdown_loader:157 - Creating local markdown loader
2026-01-14 13:16:35.015 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:35.017 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:35.019 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)


Created 7 chunks:

- ml_fundamentals_machine_learning_fundamentals: 63 chars
- ml_fundamentals_types_of_machine_learning: 58 chars
- ml_fundamentals_supervised_learning: 111 chars
- ml_fundamentals_unsupervised_learning: 119 chars
- ml_fundamentals_reinforcement_learning: 116 chars
- ml_fundamentals_model_evaluation: 252 chars
- ml_fundamentals_best_practices: 147 chars


## Create Groq Generator

The generator reads the API key from the `GROQ_API_KEY` environment variable.

In [4]:
# Create Groq generator
generator = create_groq_generator(
    model_name="llama-3.1-8b-instant",
    temperature=0.4,
    max_tokens=2048,
    use_structured_output=True,
)

print(f"Groq generator created with model: {generator.model_name}")

2026-01-14 13:16:35.423 | INFO     | fair_forge.generators:create_groq_generator:130 - Creating Groq generator with model: llama-3.1-8b-instant
2026-01-14 13:16:35.792 | INFO     | fair_forge.generators.groq_generator:__init__:73 - Initializing Groq generator with model: llama-3.1-8b-instant


Groq generator created with model: llama-3.1-8b-instant


## Generate Test Dataset

Groq's fast inference makes generation very quick!

In [5]:
import time

async def generate_dataset():
    print("Generating test dataset with Groq...\n")
    
    start_time = time.time()
    
    # generate_dataset now returns list[Dataset]
    datasets = await generator.generate_dataset(
        context_loader=loader,
        source=str(sample_file),
        assistant_id="ml-assistant",
        num_queries_per_chunk=3,
        language="english",
    )
    
    elapsed = time.time() - start_time
    
    # With default SequentialStrategy, we get one dataset
    dataset = datasets[0]
    
    print(f"Generated {len(datasets)} dataset(s) in {elapsed:.2f} seconds:")
    print(f"  Session ID: {dataset.session_id}")
    print(f"  Total queries: {len(dataset.conversation)}\n")
    
    print("Generated queries:")
    for batch in dataset.conversation:
        difficulty = batch.agentic.get('difficulty', 'N/A')
        query_type = batch.agentic.get('query_type', 'N/A')
        print(f"  [{batch.qa_id}] ({difficulty}/{query_type})")
        print(f"    {batch.query}\n")
    
    return datasets

# Execute
datasets = await generate_dataset()

2026-01-14 13:16:36.040 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:353 - Loading context from: ml_fundamentals.md
2026-01-14 13:16:36.042 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:36.043 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:36.044 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)
2026-01-14 13:16:36.045 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:355 - Loaded 7 chunks from source
2026-01-14 13:16:36.045 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:359 - Using chunk selection strategy: SequentialStrategy()
2026-01-14 13:16:36.045 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 3 queries for chunk ml_fundamentals_

Generating test dataset with Groq...



2026-01-14 13:16:37,145 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:37.178 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 3 queries for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:37.179 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 3 queries for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:37,753 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:37.757 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 3 queries for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:37.758 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 3 queries for chunk ml_fundamentals_supervised_learning
2026-01-14 13:16:38,471 - httpx - INFO - HTTP Reques

Generated 1 dataset(s) in 4.90 seconds:
  Session ID: 4a30bc8f-fe9c-4977-b6b4-7d1e5afe5612
  Total queries: 21

Generated queries:
  [ml_fundamentals_machine_learning_fundamentals_q1] (medium/factual)
    What are the fundamental concepts of machine learning?

  [ml_fundamentals_machine_learning_fundamentals_q2] (hard/application)
    How can machine learning be applied in real-world scenarios?

  [ml_fundamentals_machine_learning_fundamentals_q3] (medium/comparative)
    What are the key differences between supervised and unsupervised learning?

  [ml_fundamentals_types_of_machine_learning_q1] (easy/factual)
    What are the three main types of machine learning?

  [ml_fundamentals_types_of_machine_learning_q2] (medium/inferential)
    How does the categorization of machine learning into three types impact its applications?

  [ml_fundamentals_types_of_machine_learning_q3] (hard/comparative)
    Compare and contrast supervised and unsupervised learning.

  [ml_fundamentals_supervised_

In [ ]:
# Save dataset to JSON
dataset = datasets[0]  # Get the first (and only) dataset
output_file = Path("./generated_tests_groq.json")
with open(output_file, "w") as f:
    json.dump(dataset.model_dump(), f, indent=2)

print(f"Dataset saved to: {output_file}")

## Generate with Seed Examples

In [6]:
async def generate_with_seeds():
    seed_examples = [
        "What is the difference between supervised and unsupervised learning?",
        "How do you prevent overfitting in a machine learning model?",
        "When should you use precision vs recall as your primary metric?",
    ]
    
    print("Generating with seed examples...\n")
    
    datasets = await generator.generate_dataset(
        context_loader=loader,
        source=str(sample_file),
        assistant_id="ml-assistant",
        num_queries_per_chunk=2,
        seed_examples=seed_examples,
    )
    
    dataset = datasets[0]
    print(f"Generated {len(dataset.conversation)} queries:")
    for batch in dataset.conversation[:5]:
        print(f"  - {batch.query}")
    
    return datasets

# Execute
datasets_with_seeds = await generate_with_seeds()

2026-01-14 13:16:40.947 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:353 - Loading context from: ml_fundamentals.md
2026-01-14 13:16:40.948 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:40.949 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:40.951 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)
2026-01-14 13:16:40.952 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:355 - Loaded 7 chunks from source
2026-01-14 13:16:40.953 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:359 - Using chunk selection strategy: SequentialStrategy()
2026-01-14 13:16:40.954 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 2 queries for chunk ml_fundamentals_

Generating with seed examples...



2026-01-14 13:16:41,441 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:41.444 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 2 queries for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:41.446 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 2 queries for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:41,952 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:41.956 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 2 queries for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:41.957 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 2 queries for chunk ml_fundamentals_supervised_learning
2026-01-14 13:16:42,464 - httpx - INFO - HTTP Reques

Generated 13 queries:
  - What are the fundamental concepts covered in this guide on machine learning?
  - How does the guide on machine learning distinguish between different types of learning algorithms?
  - What are the three main types of machine learning?
  - How does the type of machine learning impact the choice of algorithm?
  - What are the primary applications of supervised learning?


## Chunk Selection Strategies

Strategies control how chunks are selected and grouped during generation. By default, all chunks are processed sequentially into a single dataset.

### RandomSamplingStrategy

Randomly samples chunks multiple times to create diverse test datasets:

In [7]:
async def generate_with_random_sampling():
    """Generate multiple datasets using random chunk sampling."""
    
    # Create a strategy that samples 3 random chunks, 2 times
    strategy = RandomSamplingStrategy(
        num_samples=2,       # Create 2 datasets
        chunks_per_sample=3, # Each with 3 random chunks
        seed=42,             # For reproducibility
    )
    
    print(f"Strategy: {strategy}\n")
    
    datasets = await generator.generate_dataset(
        context_loader=loader,
        source=str(sample_file),
        assistant_id="ml-assistant",
        num_queries_per_chunk=2,
        selection_strategy=strategy,
    )
    
    print(f"Generated {len(datasets)} datasets:\n")
    for i, ds in enumerate(datasets):
        print(f"Dataset {i+1}:")
        print(f"  Session: {ds.session_id[:8]}...")
        print(f"  Queries: {len(ds.conversation)}")
        # Show chunk IDs from the queries
        chunk_ids = set(b.agentic.get('chunk_id', 'N/A') for b in ds.conversation)
        print(f"  Chunks: {chunk_ids}\n")
    
    return datasets

# Execute
random_datasets = await generate_with_random_sampling()

2026-01-14 13:16:44.632 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:353 - Loading context from: ml_fundamentals.md
2026-01-14 13:16:44.635 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:44.636 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:44.637 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)
2026-01-14 13:16:44.638 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:355 - Loaded 7 chunks from source
2026-01-14 13:16:44.639 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:359 - Using chunk selection strategy: RandomSamplingStrategy(num_samples=2, chunks_per_sample=3, seed=42)
2026-01-14 13:16:44.640 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - 

Strategy: RandomSamplingStrategy(num_samples=2, chunks_per_sample=3, seed=42)



2026-01-14 13:16:45,126 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:45.129 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 2 queries for chunk ml_fundamentals_model_evaluation
2026-01-14 13:16:45.130 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 2 queries for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:45,639 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:45.642 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:246 - Generated 2 queries for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:45.644 | DEBUG    | fair_forge.generators.langchain_generator:generate_queries:215 - Generating 2 queries for chunk ml_fundamentals_best_practices
2026-01-14 13:16:46,151 - httpx - INFO - HTTP Request: POST ht

Generated 2 datasets:

Dataset 1:
  Session: d7367f1c...
  Queries: 6
  Chunks: {'ml_fundamentals_best_practices', 'ml_fundamentals_machine_learning_fundamentals', 'ml_fundamentals_model_evaluation'}

Dataset 2:
  Session: 79b60cfb...
  Queries: 5
  Chunks: {'ml_fundamentals_types_of_machine_learning', 'ml_fundamentals_supervised_learning', 'ml_fundamentals_model_evaluation'}



## Conversation Mode

Instead of generating independent queries, conversation mode creates coherent multi-turn conversations where each question builds on the previous ones:

In [8]:
async def generate_conversations():
    """Generate coherent multi-turn conversations."""
    
    print("Generating conversations (each turn builds on the previous)...\n")
    
    datasets = await generator.generate_dataset(
        context_loader=loader,
        source=str(sample_file),
        assistant_id="ml-assistant",
        num_queries_per_chunk=3,  # 3-turn conversations
        conversation_mode=True,   # Enable conversation mode
    )
    
    dataset = datasets[0]
    print(f"Generated {len(dataset.conversation)} conversation turns:\n")
    
    # Group by chunk to show conversation flow
    current_chunk = None
    for batch in dataset.conversation:
        chunk_id = batch.agentic.get('chunk_id', 'N/A')
        turn_num = batch.agentic.get('turn_number', 0)
        builds_on = batch.agentic.get('builds_on', None)
        
        if chunk_id != current_chunk:
            print(f"\n--- Conversation for chunk: {chunk_id} ---")
            current_chunk = chunk_id
        
        print(f"  Turn {turn_num}: {batch.query}")
        if builds_on:
            print(f"         (builds on: {builds_on})")
    
    return datasets

# Execute
conversation_datasets = await generate_conversations()

2026-01-14 13:16:47.707 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:353 - Loading context from: ml_fundamentals.md
2026-01-14 13:16:47.710 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:47.710 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:47.712 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)
2026-01-14 13:16:47.713 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:355 - Loaded 7 chunks from source
2026-01-14 13:16:47.714 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:359 - Using chunk selection strategy: SequentialStrategy()
2026-01-14 13:16:47.715 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:275 - Generating 3-turn conversation for chunk m

Generating conversations (each turn builds on the previous)...



2026-01-14 13:16:48,404 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:48.407 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:313 - Generated 3 turns for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:48.409 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:275 - Generating 3-turn conversation for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:49,121 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:49.125 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:313 - Generated 3 turns for chunk ml_fundamentals_types_of_machine_learning
2026-01-14 13:16:49.126 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:275 - Generating 3-turn conversation for chunk ml_fundamentals_supervised_learning
2026-01-14 13:16

Generated 21 conversation turns:


--- Conversation for chunk: ml_fundamentals_machine_learning_fundamentals ---
  Turn 1: What is machine learning?
  Turn 2: Can you give an example of how machine learning is used?
         (builds on: ml_fundamentals_machine_learning_fundamentals_t1)
  Turn 3: How does machine learning differ from traditional programming?
         (builds on: ml_fundamentals_machine_learning_fundamentals_t2)

--- Conversation for chunk: ml_fundamentals_types_of_machine_learning ---
  Turn 1: What are the main types of machine learning?
  Turn 2: Can you explain supervised learning in more detail?
         (builds on: ml_fundamentals_types_of_machine_learning_t1)
  Turn 3: How is supervised learning used in real-world applications?
         (builds on: ml_fundamentals_types_of_machine_learning_t2)

--- Conversation for chunk: ml_fundamentals_supervised_learning ---
  Turn 1: What is the main purpose of using labeled training data?
  Turn 2: How does the model predict 

### Combined: Random Sampling + Conversation Mode

You can combine strategies with conversation mode to create diverse conversation-based test sets:

In [9]:
async def generate_random_conversations():
    """Combine random sampling with conversation mode."""
    
    strategy = RandomSamplingStrategy(
        num_samples=2,
        chunks_per_sample=2,
        seed=42,
    )
    
    print("Generating 2 datasets with 2 random chunks each (conversation mode)...\n")
    
    datasets = await generator.generate_dataset(
        context_loader=loader,
        source=str(sample_file),
        assistant_id="ml-assistant",
        num_queries_per_chunk=2,  # 2-turn conversations
        selection_strategy=strategy,
        conversation_mode=True,
    )
    
    for i, ds in enumerate(datasets):
        print(f"Dataset {i+1} ({len(ds.conversation)} turns):")
        for batch in ds.conversation[:4]:  # Show first 4 turns
            chunk = batch.agentic.get('chunk_id', 'N/A')[:15]
            turn = batch.agentic.get('turn_number', 0)
            print(f"  [{chunk}] T{turn}: {batch.query[:50]}...")
        print()
    
    return datasets

# Execute
combined_datasets = await generate_random_conversations()

2026-01-14 13:16:52.823 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:353 - Loading context from: ml_fundamentals.md
2026-01-14 13:16:52.825 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:275 - Loading 1 markdown file(s)
2026-01-14 13:16:52.826 | INFO     | fair_forge.generators.context_loaders.local_markdown:_load_single_file:137 - Loading markdown file: ml_fundamentals.md
2026-01-14 13:16:52.828 | INFO     | fair_forge.generators.context_loaders.local_markdown:load:282 - Created 7 total chunks from 1 file(s)
2026-01-14 13:16:52.828 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:355 - Loaded 7 chunks from source
2026-01-14 13:16:52.829 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:359 - Using chunk selection strategy: RandomSamplingStrategy(num_samples=2, chunks_per_sample=2, seed=42)
2026-01-14 13:16:52.830 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:2

Generating 2 datasets with 2 random chunks each (conversation mode)...



2026-01-14 13:16:53,318 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:53.323 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:313 - Generated 2 turns for chunk ml_fundamentals_model_evaluation
2026-01-14 13:16:53.324 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:275 - Generating 2-turn conversation for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:53,933 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-14 13:16:53.938 | DEBUG    | fair_forge.generators.langchain_generator:generate_conversation:313 - Generated 2 turns for chunk ml_fundamentals_machine_learning_fundamentals
2026-01-14 13:16:53.939 | INFO     | fair_forge.generators.langchain_generator:generate_dataset:414 - Generated 4 batches for chunk group (2 chunks)
2026-01-14 13:16:53.941 | DEBUG    | fair_forge.generato

Dataset 1 (4 turns):
  [ml_fundamentals] T1: What are the key metrics used to evaluate ML model...
  [ml_fundamentals] T2: Can you explain the difference between accuracy an...
  [ml_fundamentals] T1: What is machine learning?...
  [ml_fundamentals] T2: Can you give an example of how machine learning is...

Dataset 2 (4 turns):
  [ml_fundamentals] T1: What is machine learning?...
  [ml_fundamentals] T2: How does it differ from traditional programming?...
  [ml_fundamentals] T1: What metrics are commonly used to evaluate ML mode...
  [ml_fundamentals] T2: Can you explain the difference between precision a...



In [20]:
# Save dataset to JSON
combined_dataset = combined_datasets[0] 
output_file = Path("./combined_dataset_groq.json")
with open(output_file, "w") as f:
    json.dump(combined_dataset.model_dump(), f, indent=2)

print(f"Dataset saved to: {output_file}")

Dataset saved to: combined_dataset_groq.json


## Available Groq Models

Check them [here](https://console.groq.com/docs/models).

## Cleanup